# **Program to extract external hyperlinks on the target Home URL**

# Extract external hyperlinks

In [1]:
# Install requests for web service
!pip install requests

In [2]:
# Library needed
# Regular Python Data analyze library
import pandas as pd

# Web related library
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

In [ ]:
# Replace the home_url here
Home_url = ""

In [4]:
# User Agent to prevent being identified as a bot
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [5]:
MAX_PAGES = 10000
# Google Colab Memory Exceed
MAX_QUEUE = 20000
MAX_DEPTH = 5  # Limit depth to 3

# Function to extract all external links
# From the starting homepage
def extract_links(start_url):
  to_visit = [(start_url, 0)]  # Queue of (url, depth)

  # While queue is not empty
  while to_visit:
    # extract homepage link and current depth
    url, depth = to_visit.pop(0)

    # Check if we've hit the page limit
    if len(visited_urls) >= MAX_PAGES or len(to_visit) >= MAX_QUEUE:
      break

    # Skip if the URL has already been visited
    if url in visited_urls or depth > MAX_DEPTH:
      continue

    # First attempt with User-Agent header
    try:
      response = requests.get(url, headers=headers, timeout=10)
      response.encoding = 'utf-8'  # explicitly set the encoding
      soup = BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
      print(f"Failed to access {url} with User-Agent: {e}, attempting without User-Agent")
      # Second attempt without User-Agent
      try:
        response = requests.get(url, timeout=10)
        response.encoding = 'utf-8'  # explicitly set the encoding
        soup = BeautifulSoup(response.content, 'html.parser')
      except requests.RequestException as e:
        print(f"Failed to access {url} without User-Agent: {e}, skipping this URL")
        continue

    # Mark the URL as visited
    visited_urls.add(url)
    # print(f"Visiting Internal Url (Depth {depth}): {url}")

    # Get the base domain and path of the URL
    base_domain = urlparse(url).netloc
    base_path = urlparse(url).path

    # Find all links on the page
    links = soup.find_all('a')

    # Go through all links in the list
    for link in links:
      href = link.get('href')
      if href:
        full_url = urljoin(url, href)
        parsed_url = urlparse(full_url)

        # If it's an external link, add to external_links set
        if parsed_url.netloc != base_domain:
          external_links.add(full_url)
        else:
          # If it's an internal link, and not yet visited, add to the queue
          # Make sure the base domain and base path both matches
          if parsed_url.netloc == base_domain and parsed_url.path.startswith(base_path):
            if full_url not in visited_urls and not full_url.endswith((".pdf", ".jpg", ".png", ".gif", ".zip")):
              # Ensure the full_url is not in the to_visit queue with any depth
              if not any(url == full_url for url, _ in to_visit):
                # Check if the URL contains "sponsor" or "partner"
                if "Sponsor" in full_url or "sponsor" in full_url or "Partner" in full_url or "partner" in full_url or "Funder" in full_url or "funder" in full_url or "Donor" in full_url or "donor" in full_url:
                    to_visit.insert(0, (full_url, depth + 1))  # Add to the front of the queue
                else:
                    to_visit.append((full_url, depth + 1))  # Add to the end of the queue

In [6]:
# Click Run, and you should be able to see the internal links the cralwer visting
# The program running time highly depends on the website size
# You can run this multiple times to ensure all corners of the homepage has been visited

# Track visited internal URLs and external links
visited_urls = set()
external_links = set()
# Get all the external links on the web of this sponsor
extract_links(Home_url)

In [ ]:
# View the external links
# The ones ends with ".com", ".org" etc are usually more interesting
external_links